Rodei isso aqui para conseguir testBooks e trainBooks

In [1]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('./goodbooksData/ratings.csv')
df

,book_id,user_id,rating,original_title
0,1,314,5,Harry Potter and the Half-Blood Prince
1,1,439,3,Harry Potter and the Half-Blood Prince
2,1,588,5,Harry Potter and the Half-Blood Prince
3,1,1169,4,Harry Potter and the Half-Blood Prince
4,1,1185,4,Harry Potter and the Half-Blood Prince
...,...,...,...,...
64283,9998,51295,5,砂の女 [Suna no onna]
64284,9998,51559,5,砂の女 [Suna no onna]
64285,9998,52087,4,砂の女 [Suna no onna]
64286,9998,52330,4,砂の女 [Suna no onna]


In [2]:
duplicates = df[df.duplicated(['book_id', 'user_id'], keep=False)]
duplicates

,book_id,user_id,rating,original_title
1073,25,32635,5,Notes from a Big Country
1074,25,32635,5,Notes from a Big Country
1317,28,9011,2,Notes from a Small Island
1318,28,9011,1,Notes from a Small Island
1546,30,24582,4,The Hobbit and The Lord of the Rings
...,...,...,...,...
63395,9833,46077,4,"めくらやなぎと、眠る女 [Mekurayanagi to, nemuru onna]"
63720,9865,38053,4,The Moor's Last Sigh
63721,9865,38053,3,The Moor's Last Sigh
63988,9915,39628,4,Less Than Zero


In [3]:
df = df.drop_duplicates(subset=['book_id', 'user_id'], keep='first')
df

,book_id,user_id,rating,original_title
0,1,314,5,Harry Potter and the Half-Blood Prince
1,1,439,3,Harry Potter and the Half-Blood Prince
2,1,588,5,Harry Potter and the Half-Blood Prince
3,1,1169,4,Harry Potter and the Half-Blood Prince
4,1,1185,4,Harry Potter and the Half-Blood Prince
...,...,...,...,...
64283,9998,51295,5,砂の女 [Suna no onna]
64284,9998,51559,5,砂の女 [Suna no onna]
64285,9998,52087,4,砂の女 [Suna no onna]
64286,9998,52330,4,砂の女 [Suna no onna]


In [4]:
user_counts = df['user_id'].value_counts()
users_to_keep = user_counts[user_counts > 1].index

df = df[df['user_id'].isin(users_to_keep)]
df

,book_id,user_id,rating,original_title
0,1,314,5,Harry Potter and the Half-Blood Prince
1,1,439,3,Harry Potter and the Half-Blood Prince
2,1,588,5,Harry Potter and the Half-Blood Prince
3,1,1169,4,Harry Potter and the Half-Blood Prince
4,1,1185,4,Harry Potter and the Half-Blood Prince
...,...,...,...,...
64276,9998,44360,4,砂の女 [Suna no onna]
64277,9998,45031,4,砂の女 [Suna no onna]
64278,9998,47437,4,砂の女 [Suna no onna]
64285,9998,52087,4,砂の女 [Suna no onna]


In [5]:
grouped = df.groupby('user_id')

train_df = pd.DataFrame(columns=df.columns)
test_df = pd.DataFrame(columns=df.columns)

for user_id, group in tqdm(grouped, desc="Processing users"):
    test_sample = group.sample(n=1, random_state=42)
    train_sample = group.drop(test_sample.index)
    test_df = pd.concat([test_df, test_sample], ignore_index=True)
    train_df = pd.concat([train_df, train_sample], ignore_index=True)

Processing users: 100%|██████████| 13115/13115 [00:09<00:00, 1437.04it/s]


In [6]:
train_df

,book_id,user_id,rating,original_title
0,1519,7,5,Ὀρέστεια
1,4138,7,3,Naked
2,4588,7,3,Extremely Loud and Incredibly Close
3,5084,10,2,My Life in France
4,8698,23,4,"So Long, and Thanks for All the Fish"
...,...,...,...,...
38887,629,53403,2,Zen and the Art of Motorcycle Maintenance
38888,2187,53403,5,Middlesex
38889,3764,53403,5,"From Russia, with Love"
38890,8682,53403,4,Jitterbug Perfume


In [7]:
test_df


,book_id,user_id,rating,original_title
0,3711,7,5,White Teeth
1,5907,10,4,The Hobbit or There and Back Again
2,5161,23,3,The Deep End of the Ocean
3,5553,27,3,Six Easy Pieces: Essentials of Physics Explain...
4,249,35,4,Tropic of Cancer
...,...,...,...,...
13110,3473,53378,2,A Walk to Remember
13111,1715,53381,4,Metamorphoses
13112,6295,53393,4,Howl and Other Poems
13113,1624,53403,4,West with the Night


In [8]:
test_value_counts = test_df['user_id'].value_counts()
test_unique_counts = test_df['user_id'].nunique()

print(test_value_counts)
print(test_unique_counts)

user_id
7        1
34029    1
33995    1
33996    1
33998    1
        ..
16675    1
16688    1
16701    1
16702    1
53420    1
Name: count, Length: 13115, dtype: int64
13115


In [9]:
train_df.to_csv('train_v2.csv', index=False)
test_df.to_csv('test_v2.csv', index=False)